In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [7]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [53]:
# 初始化构造数据
from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
build_pos_train_and_valid_data()



195 21700 0.008986175115207374
274 31200 0.008782051282051282
276 31700 0.008706624605678233
307 36100 0.00850415512465374
327 36600 0.008934426229508196
356 37500 0.009493333333333333
435 46200 0.009415584415584415
624 67800 0.009203539823008849
724 76800 0.009427083333333334
914 98800 0.009251012145748989
1091 120300 0.009068994181213633
1204 134500 0.008951672862453531
1397 156900 0.008903760356915232
1415 158700 0.008916194076874606
1725 203000 0.008497536945812808
1925 258700 0.007441051410900657
1991 287300 0.0069300382875043505
2111 336000 0.006282738095238096
2142 348200 0.006151636990235497
2288 419300 0.005456713570236108
2292 793378 0.0028889129771685123


In [11]:
import torch
torch.cuda.set_device(1)

In [ ]:
# 正常训练

# from bailian_nlp.modules import BertData

# data = BertData.create(
#     train_path,
#     valid_path, 
#     vocab_file,
#     data_type="bert_uncased",
#     max_seq_len=128,
#     batch_size=32
    
# )

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256,
    freeze=False
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 1
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


INFO:root:Loading pretrained bert model!
INFO:root:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:root:Resuming train... Current epoch 0.


In [39]:
num_epochs = 1

learner.fit(num_epochs, target_metric='f1')

INFO:root:Resuming train... Current epoch 16.


In [54]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
# learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


INFO:bailian_nlp.released.pos:load default user_dict in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/dict/user_dict.txt
INFO:bailian_nlp.released.dictionary:本次加载词条数：3
INFO:bailian_nlp.released.dictionary:当前总词条数: 3
INFO:bailian_nlp.released.pos:found pos model file in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
INFO:bailian_nlp.released.pos:pos model loads success!


DEBUG:bailian_nlp.web.utils.common:get_data cost 93.030708s


DEBUG:bailian_nlp.web.utils.common:get_data cost 0.215367s


INFO:bailian_nlp.released.pos:found pos model file in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
INFO:bailian_nlp.released.pos:pos model loads success!
INFO:root:Resuming train... Current epoch 0.


In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [4]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


INFO:bailian_nlp.released.pos:load default user_dict in /Users/lx/PycharmProjects/pos-bert/bailian_nlp/datadrive/dict/user_dict.txt
INFO:bailian_nlp.released.dictionary:本次加载词条数：4
INFO:bailian_nlp.released.dictionary:当前总词条数: 4
INFO:bailian_nlp.released.pos:found pos model file in /Users/lx/PycharmProjects/pos-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
INFO:bailian_nlp.released.pos:pos model loads success!


In [16]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
# text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
# text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
# text = '董事'
# text = '一言不合'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
# text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
# text = 'www.baidu.com'
# text = '2015-10-8   ”的维修保养和改造升级服务。机场客户包括上海虹桥、深圳宝安、长沙黄花、武汉天河、郑州新郑、重庆江北、西安咸阳、大连周水子、南京禄口、杭州萧山等'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

DEBUG:bailian_nlp.web.utils.common:get_data cost 0.026153s



0.2881739139556885


[[('美年大健康产业（集团）有限公司', 'nt'),
  ('美年大健康产业（集团）有限公司', 'nt'),
  ('美年大健康产业（集团）有限公司', 'nt'),
  ('始创', 'v'),
  ('于', 'p'),
  ('2004年', 't'),
  (',', 'w'),
  ('是', 'v'),
  ('中国健康体检和医疗服务集团', 'nt'),
  (',', 'w'),
  ('总部', 'n'),
  ('位于', 'v'),
  ('上海', 'ns'),
  (',', 'w'),
  ('深耕', 'v'),
  ('布局', 'v'),
  ('北京', 'ns'),
  ('、', 'w'),
  ('深圳', 'ns'),
  ('、', 'w'),
  ('沈阳', 'ns'),
  ('、', 'w'),
  ('广州', 'ns'),
  ('、', 'w'),
  ('成都', 'ns'),
  ('、', 'w'),
  ('武汉', 'ns'),
  ('、', 'w'),
  ('.', 'w'),
  ('.', 'w'),
  ('.', 'w')]]